In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema_source", "bronze_jptq")
dbutils.widgets.text("esquema_stage", "silver_jptq")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_stage = dbutils.widgets.get("esquema_stage")

In [0]:
def fn_transform_pelicula(catalogo: str, esquema_source: str):
   sql_select_pelicula = f"""SELECT
                              P.ID_PELICULA,
                              P.DES_PELICULA,
                              MIP.DES_DURACION_PELICULA,
                              DP.DES_TOP_ACTORES,
                              PP.DES_RUTA_POSTER_FRONTAL,
                              PP.DES_RUTA_POSTER_TRASERA,
                              P.FEC_LANZAMIENTO
                              FROM {catalogo}.{esquema_source}.TBL_PELICULAS P
                              LEFT JOIN {catalogo}.{esquema_source}.TBL_DETALLE_PELICULAS DP
                              ON P.ID_PELICULA = DP.ID_PELICULA
                              LEFT JOIN {catalogo}.{esquema_source}.TBL_MAS_INFO_PELICULAS MIP
                              ON P.ID_PELICULA = MIP.PELICULA_ID
                              LEFT JOIN {catalogo}.{esquema_source}.TBL_POSTER_PELICULAS PP
                              ON P.ID_PELICULA = PP.ID_PELICULA
                              GROUP BY P.ID_PELICULA,P.DES_PELICULA,MIP.DES_DURACION_PELICULA,
                              DP.DES_TOP_ACTORES,PP.DES_RUTA_POSTER_FRONTAL,PP.DES_RUTA_POSTER_TRASERA,
                              P.FEC_LANZAMIENTO
                              ORDER BY P.ID_PELICULA ASC"""
   df_t_pelicula = spark.sql(sql_select_pelicula)
   return df_t_pelicula

In [0]:
df_tmp_pelicula = fn_transform_pelicula(catalogo, esquema_source)

In [0]:
df_tmp_pelicula = df_tmp_pelicula.withColumn("FEC_CARGA", current_timestamp())

In [0]:
display(df_tmp_pelicula)

In [0]:
df_tmp_pelicula.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_stage}.TBL_TMP_PELICULA")

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.silver_jptq.tbl_tmp_pelicula